# 모델 설명

좋아, 팀원들이 쉽게 이해할 수 있도록 **`perfume_model_api.ipynb` 전체 구조**를 각 블록마다 설명을 붙여 정리해줄게. 이건 마크다운 셀로 노트북에 붙이면 그대로 사용 가능해.

---

# 💡 향수 추천 모델 전체 구조 설명 (`perfume_model_api.ipynb`)

## 1. ✅ 라이브러리 임포트 및 재현성 고정

```python
import os, random, pickle
import numpy as np
...
tf.random.set_seed(42)
```

* 데이터 처리 및 모델 학습에 필요한 주요 라이브러리를 불러오고,
* 결과의 **일관성**을 위해 seed를 고정합니다.

---

## 2. 📦 데이터 로딩 및 전처리

```python
df = pd.read_csv('./data/dataset.csv')
...
df['notes'] = df['notes'].apply(clean_notes)
```

* `.csv` 파일에서 향수 데이터를 불러오고,
* `notes` 필드를 정제합니다 (불필요한 기호/공백 제거 등).

---

## 3. 🧪 향료 벡터화

```python
note_vectorizer = CountVectorizer(token_pattern=r'[^,]+')
...
note_df = pd.DataFrame(...)
```

* `notes`(향료 목록)를 Bag-of-Words 방식으로 벡터화합니다.
* 각 향수마다 향료 출현 여부가 벡터로 표현됩니다.

---

## 4. 🧠 입력값 인코딩

```python
encoder = OrdinalEncoder()
...
X = encoder.transform(...)
```

* 성별, 계절, 시간대 등의 **카테고리 특성**을 숫자로 인코딩합니다.
* OneHot이 아닌 **OrdinalEncoder**를 사용해 의미론적 거리 반영.

---

## 5. 🧩 학습 데이터 분할 및 클래스 가중치 계산

```python
X_train, X_val, y_train, y_val = ...
class_weights = compute_class_weight(...)
```

* 전체 데이터를 **학습/검증**으로 나누고,
* 불균형한 감정 클래스 문제를 해결하기 위해 **클래스별 가중치**를 계산합니다.

---

## 6. 🏗️ 모델 정의 및 학습

```python
model = Sequential([...])
model.compile(...)
model.fit(...)
```

* 심플한 Dense(64) 구조의 **MLP 모델**을 정의합니다.
* `EarlyStopping`으로 과적합 방지, 클래스 가중치 적용.

---

## 7. 🧾 모델 성능 평가

```python
y_pred = ...
results = {
    "classification_report": ...,
    "macro_f1": ...,
    "weighted_f1": ...
}
```

* 검증 데이터를 이용해 **정밀도, 재현율, F1 점수**를 계산합니다.
* 추후 분석을 위한 JSON 형태로 저장.

---

## 8. 💾 모델 및 도구 저장

```python
model.save(...)
pickle.dump(...)
note_df.to_csv(...)
```

* 학습이 완료된 모델 및 인코더, 벡터라이저를 `.keras`, `.pkl`, `.csv` 형태로 저장.
* API 서버에서 바로 로딩 가능하게 구성됨.

---

## 9. 🔍 감정 예측 함수 (`predict_emotion`)

```python
def predict_emotion(user_input):
    ...
    return {
        "cluster": pred,
        "description": EMOTION_DESC[pred],
        "proba": proba.tolist()
    }
```

* 사용자의 태그 입력을 바탕으로 **감정 클러스터를 예측**하고,
* 감정 설명(`description`)과 확률 분포(`proba`)를 반환.

---

## 10. 📝 주요 노트 추출 함수 (`extract_notes`)

```python
def extract_notes(...):
    ...
    return 상위 노트 리스트, 1차 추천 향수 인덱스
```

* 예측된 감정 클러스터에 기반해 **대표적인 노트 15개**를 추출.
* 향수 벡터 간 유사도를 활용해 **중복되지 않는 향수 10개**도 선택.

---

## 11. 🌸 향수 재추천 함수 (`recommend_perfumes`)

```python
def recommend_perfumes(...):
    ...
    return 추천 향수 리스트 (JSON 딕셔너리)
```

* 사용자가 선택한 **노트 선호도 점수**를 받아,
* 감정 + 노트 기반의 `final_score`로 2차 향수 추천을 수행.

---

## ✅ 연동 포인트 요약

| 목적        | 설명                                          |
| --------- | ------------------------------------------- |
| 프론트 입력    | `user_input`: 태그 6개                         |
| 1차 호출 API | `/predict_emotion`: 감정 클러스터 + 설명 + proba 반환 |
| 노트 선택 이후  | `/recommend_perfumes`: 추천 향수 10개 반환         |
| 결과 포맷     | `JSON` 형태로 API 응답 가능                        |

---

필요하면 이 설명을 `.ipynb` 내부 마크다운 셀로 만들어서 추가해줄 수 있어. 원할 때 말해줘!


# 모델

In [ ]:

# perfume_model_api.ipynb
# ✅ API 연동용으로 리팩토링됨

import os, random, pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

# 재현성 고정
random.seed(42)
np.random.seed(42)
os.environ["PYTHONHASHSEED"] = str(42)
tf.random.set_seed(42)

# 데이터 로드 및 전처리
df = pd.read_csv('./data/dataset.csv')
df['notes'] = df['notes'].fillna('').str.lower()

def clean_notes(raw_notes):
    notes = [n.strip() for n in raw_notes.split(',')]
    return ', '.join([n for n in notes if len(n) > 0 and len(n) < 40])

df['notes'] = df['notes'].apply(clean_notes)

note_vectorizer = CountVectorizer(token_pattern=r'[^,]+')
note_matrix = note_vectorizer.fit_transform(df['notes'])
note_df = pd.DataFrame(note_matrix.toarray(), columns=note_vectorizer.get_feature_names_out())

# 인코딩 및 분할
encoder = OrdinalEncoder()
X_input = df[['gender', 'season_tags', 'time_tags', 'desired_impression', 'activity', 'weather']]
encoder.fit(X_input.values)
X = encoder.transform(X_input.values)
y = df['emotion_cluster']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: w for i, w in zip(np.unique(y_train), class_weights)}

# 모델 정의
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(6, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, callbacks=[early_stop], class_weight=class_weight_dict)

# 성능 평가
y_pred = model.predict(X_val).argmax(axis=1)
results = {
    "classification_report": classification_report(y_val, y_pred, output_dict=True),
    "macro_f1": f1_score(y_val, y_pred, average='macro'),
    "weighted_f1": f1_score(y_val, y_pred, average='weighted')
}
results

# 저장
model.save('./models/final_model.keras')
with open('./models/encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)
with open('./models/note_vectorizer.pkl', 'wb') as f:
    pickle.dump(note_vectorizer, f)
note_df.to_csv('./data/note_df.csv', index=False)

# 추론 함수 정의
EMOTION_DESC = {
    0: "따뜻하고 친근한 감정",
    1: "신선하고 활기찬 느낌",
    2: "우아하고 세련된 분위기",
    3: "관능적이고 매혹적인 향",
    4: "부드럽고 순수한 감정",
    5: "신비롭고 독특한 감정"
}

def predict_emotion(user_input):
    encoder = pickle.load(open('./models/encoder.pkl', 'rb'))
    model = load_model('./models/final_model.keras')
    user_vec = encoder.transform([user_input])
    proba = model.predict(user_vec)[0]
    pred = int(np.argmax(proba))
    return {
        "cluster": pred,
        "description": EMOTION_DESC[pred],
        "proba": proba.tolist()
    }

def extract_notes(df, note_df, proba):
    df['emotion_score'] = df['emotion_cluster'].map(lambda c: proba[c])
    selected = []
    top_sorted = df.sort_values('emotion_score', ascending=False)
    for i in top_sorted.index:
        if all(cosine_similarity([note_df.loc[i]], [note_df.loc[j]])[0][0] < 0.95 for j in selected):
            selected.append(i)
        if len(selected) == 10:
            break
    top_perfumes = df.loc[selected]
    top_notes_matrix = note_df.loc[top_perfumes.index]
    top_notes_sum = top_notes_matrix.sum(axis=0)
    return top_notes_sum.sort_values(ascending=False).head(15).index.tolist(), selected

def recommend_perfumes(df, note_df, user_note_scores, selected_idx, proba):
    user_note_vec = np.zeros((1, len(note_df.columns)))
    for i, note in enumerate(note_df.columns):
        score = user_note_scores.get(note, 0)
        user_note_vec[0, i] = score / 5
    note_cos_sim = cosine_similarity(note_df.values, user_note_vec).reshape(-1)
    note_sum = np.zeros(len(note_df))
    for note, weight in user_note_scores.items():
        if note in note_df.columns:
            note_sum += note_df[note].to_numpy().ravel() * weight
    note_score = 0.7 * note_cos_sim + 0.3 * (note_sum / 10)
    df['note_score'] = note_score
    df['is_top10'] = df.index.isin(selected_idx).astype(int)
    df['emotion_score'] = df['emotion_cluster'].map(lambda c: proba[c])
    df['final_score'] = 0.7 * df['emotion_score'] + 0.25 * df['note_score'] + 0.05 * df['is_top10']
    df['note_diversity'] = note_df.astype(bool).sum(axis=1)
    top10 = df.sort_values(by=['final_score', 'note_diversity'], ascending=[False, False]).head(10)
    return top10[['name', 'brand', 'final_score', 'emotion_cluster']].to_dict(orient='records')
